In [1]:
import heapq as hq
import pandas as pd

import warnings
warnings.filterwarnings('ignore')

In [2]:
# test case 1
processes = ['P1','P2','P3']
arrival_time = [1,0,1]
burst_time = [10,5,8]
waiting_time = [0,18,10]
turnaround_time = [10,23,18]
complete_time = [11,23,19]
priority_num = [1,3,2]

In [14]:
# test case 2
processes = ['P1','P2','P3','P4']
arrival_time = [0,2,4,5]
burst_time = [3,2,1,4]
waiting_time = [3,0,0,1]
turnaround_time = [6,2,1,5]
complete_time = [6,4,5,10]
priority_num = [3,1,2,4]

In [15]:
expected_answer = pd.DataFrame({'Process':processes, 'Arrival Time':arrival_time, 'Burst Time':burst_time, 
                               'Waiting Time':waiting_time, 'Turn Around Time':turnaround_time, 'Completion Time':complete_time, 
                               'Priority Number':priority_num})
expected_answer.sort_values(by=['Arrival Time', 'Burst Time'], ignore_index=True, inplace=True)

In [16]:
def generate_gantt_chart(timeline):
    end_at_prev = 0
    for i in timeline:
        start_at = i[0]
        end_at = i[1] + start_at
        
        if start_at != end_at_prev:
            print(f"{end_at_prev}-{start_at} : Idle | ", end="")
    
        print(f"{start_at}-{end_at} : Process {i[2]} | ", end="")
        
        end_at_prev = end_at

In [17]:
# generate a sorted timeline for the srtf algorithm

def sort_process_priority(processes, arrival_time, burst_time, priority_num):
    end_time = sum(burst_time)
    current_time = 0
    remaining_time = burst_time.copy()
    processes_list = [ list(tup) for tup in list(zip(arrival_time, priority_num, burst_time, processes, remaining_time))]
    sorted_process_list = []
    hq.heapify(processes_list)
    
    while current_time < end_time or processes_list:
        available_processes_list = [process for process in processes_list if process[0] <= current_time]
        
        if available_processes_list:
            process = min(available_processes_list, key=lambda x: x[1] )
            processes_list.remove(process)
            print("Process selected", process)
            
            process[4] = process[4] - 1
            
            if process[4] > 0:
                hq.heappush(processes_list, process)

            
            if len(sorted_process_list) == 0 or sorted_process_list[-1][2] != process[3]:
                # sorted_process_list.append( [ current_time, current_time + 1, process[2] ] ) # pocess start time, process end time, process name
                sorted_process_list.append( [ current_time, 1, process[3] ] )  # process start time, process duration, process name
            else:
                sorted_process_list[-1][1] += 1
            
            current_time +=1
            
        else:
            process = hq.nsmallest( 1, processes_list, key=lambda x: x[0] )[0]
            print("Process selected", process)
            
            process[4] = process[4] - 1
            
            if process[4] > 0:
                hq.heappush(processes_list, process)
            
            current_time = process[0]
            
            # sorted_process_list.append( [ current_time, current_time, process[2] ] )
            # sorted_process_list.append( [ current_time, current_time + 1, process[2] ] ) # pocess start time, process end time, process name
            
            sorted_process_list.append( [ current_time, 0, process[3] ] )  # process start time, process duration, process name
    
    print("\nSorted Processed List", sorted_process_list)
    return sorted_process_list

In [18]:
# writing a program for shortest job first scheduling algorithm with arrival times to return
# waiting time and turn around time for each process and average waiting time and average turn around time
# for all processes


def priority(process, arrival_time, burst_time, priority_num):    
    # for preemptive algorithms
    
    timeline = sort_process_priority(processes, arrival_time, burst_time, priority_num)
    processes_list = [ list(tup) for tup in list(zip( process, arrival_time, burst_time, priority_num )) ]
    df = pd.DataFrame(processes_list, columns=['Process', 'Arrival Time', 'Burst Time', "Priority Number"])
    timeline_df = pd.DataFrame(timeline, columns=['Start Time', 'Duration', 'Process'])
    
    list_of_processes = df['Process'].tolist()
    for i in list_of_processes:
        temp = timeline_df.loc[timeline_df['Process'] == i, 'Start Time'] + timeline_df.loc[timeline_df['Process'] == i, 'Duration']
        df.loc[df['Process'] == i, 'Completion Time'] = temp.iloc[-1]

    df['Turn Around Time'] = df['Completion Time'] - df['Arrival Time']
    df['Waiting Time'] = df['Turn Around Time'] - df['Burst Time']    
    
    df = df[['Process', 'Arrival Time', 'Burst Time', 'Waiting Time', 'Turn Around Time', 'Completion Time',"Priority Number"]]
        
    print("\nProcess Details:")
    for i in range(len(df)):
        print(f"Process: {df['Process'].iloc[i]}, Wait Time: {df['Waiting Time'].iloc[i]}, Turn Around Time: {df['Turn Around Time'].iloc[i]}, Completion Time: {df['Completion Time'].iloc[i]}")
        
    print(f"\nAverage Waiting Time: {df['Waiting Time'].mean():.2f} ")
    print(f"Average Turn Around Time:  {df['Turn Around Time'].mean():.2f} ")
    return df.reset_index(drop=True) , timeline

In [19]:
actual_answer, timeline = priority(processes, arrival_time, burst_time, priority_num)

Process selected [0, 3, 3, 'P1', 3]
Process selected [0, 3, 3, 'P1', 2]
Process selected [2, 1, 2, 'P2', 2]
Process selected [2, 1, 2, 'P2', 1]
Process selected [4, 2, 1, 'P3', 1]
Process selected [0, 3, 3, 'P1', 1]
Process selected [5, 4, 4, 'P4', 4]
Process selected [5, 4, 4, 'P4', 3]
Process selected [5, 4, 4, 'P4', 2]
Process selected [5, 4, 4, 'P4', 1]

Sorted Processed List [[0, 2, 'P1'], [2, 2, 'P2'], [4, 1, 'P3'], [5, 1, 'P1'], [6, 4, 'P4']]

Process Details:
Process: P1, Wait Time: 3.0, Turn Around Time: 6.0, Completion Time: 6.0
Process: P2, Wait Time: 0.0, Turn Around Time: 2.0, Completion Time: 4.0
Process: P3, Wait Time: 0.0, Turn Around Time: 1.0, Completion Time: 5.0
Process: P4, Wait Time: 1.0, Turn Around Time: 5.0, Completion Time: 10.0

Average Waiting Time: 1.00 
Average Turn Around Time:  3.50 


In [20]:
generate_gantt_chart(timeline)

0-2 : Process P1 | 2-4 : Process P2 | 4-5 : Process P3 | 5-6 : Process P1 | 6-10 : Process P4 | 

In [21]:
actual_answer 

,Process,Arrival Time,Burst Time,Waiting Time,Turn Around Time,Completion Time,Priority Number
0,P1,0,3,3.0,6.0,6.0,3
1,P2,2,2,0.0,2.0,4.0,1
2,P3,4,1,0.0,1.0,5.0,2
3,P4,5,4,1.0,5.0,10.0,4


In [22]:
expected_answer

,Process,Arrival Time,Burst Time,Waiting Time,Turn Around Time,Completion Time,Priority Number
0,P1,0,3,3,6,6,3
1,P2,2,2,0,2,4,1
2,P3,4,1,0,1,5,2
3,P4,5,4,1,5,10,4


In [23]:
check_actual_answer = actual_answer.sort_values(by=['Arrival Time', 'Burst Time'], ignore_index=True)

In [24]:
diff = check_actual_answer.compare(expected_answer)

if diff.empty:
    print("Your answer is correct")
else:
    print("Your answer is incorrect")
    
diff

Your answer is correct


Empty DataFrame
Columns: []
Index: []

In [25]:
# # insert this line in srtf() function before uncommenting this cell and running it
# # df = df.astype({'Completion Time': 'int64', 'Turn Around Time': 'int64', 'Waiting Time': 'int64'})

# from pandas.testing import assert_frame_equal
# assert_frame_equal(check_actual_answer, expected_answer)